In [ ]:
from torch.utils.data.dataloader import DataLoader
from data.main.data import GPTDATA
from gpt.model.gpt import GPT
import torch.nn as nn
import torch

In [ ]:
gptdata = GPTDATA()
gptdata.curate(max_length = 512)

In [ ]:
VOCAB_SIZE = gptdata.vocab_size
SEQ_LEN = gptdata.seq_len
TOKENIZER = gptdata.tokenizer_gpt

In [ ]:
data = gptdata.hf_dataset

In [ ]:
data

In [ ]:
dataloader = DataLoader(
    data,
    batch_size=30
)

In [ ]:
gptmodel = GPT(vocab_size=VOCAB_SIZE, 
               context_length=SEQ_LEN, 
               model_dim=128, 
               num_blocks=4, 
               num_heads=4)

In [ ]:
optimizer = torch.optim.SGD(gptmodel.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.CrossEntropyLoss(reduction='mean')

In [ ]:
for _ in range(100):
    running_loss = 0 
    
    ds = next(iter(dataloader))

    input_ids = torch.stack(ds['input_ids']).T
    label = torch.stack(ds['label']).T
    attn = torch.stack(ds['attention_mask']).T # We don't need this as it's already masked in selfattn.py line 23 but needed for loss.

    # Zero your gradients for every batch!
    optimizer.zero_grad()

    # Make predictions for this batch
    outputs = gptmodel(input_ids)

    # Compute the loss and its gradients
    first = True
    
    for i in range(30):
        if first:
            first=False
            loss =loss_fn(outputs[i][attn[i]==1], label[i][attn[i]==1])
        else:
            loss +=loss_fn(outputs[i][attn[i]==1], label[i][attn[i]==1])

    loss.backward()
    
    # Adjust learning weights
    optimizer.step()

    # Gather data and report
    running_loss += loss.item()

    print(running_loss)